In [ ]:
from techminer2.database.metrics.performance import DataFrame  # type: ignore


frame = (
    DataFrame()
    #
    # FIELD:
    .with_field("subject_areas")
    .having_terms_in_top(None)
    .having_terms_ordered_by("OCC")
    .having_term_occurrences_between(None, None)
    .having_term_citations_between(None, None)
    .having_terms_in(None)
    #
    # DATABASE:
    .where_root_directory_is("../")
    .where_database_is("main")
    .where_record_years_range_is(None, None)
    .where_record_citations_range_is(None, None)
    #
    .run()
)

frame["percentage"] = frame["OCC"] / 15286.0 * 100
frame["percentage"] = frame["percentage"].map(lambda x: "{:2.1f}%".format(x))
frame = frame[["OCC", "percentage", "global_citations", "local_citations"]]
frame

In [ ]:
from techminer2.database.metrics.records import DataFrame # type: ignore


for subject_area in frame.index:

    frame_by_area = (
        DataFrame()
        #
        .where_root_directory_is("../")
        .where_database_is("main")
        .where_record_years_range_is(None, None)
        .where_record_citations_range_is(None, None)
        .where_records_match(
            {
                "subject_areas": [subject_area],
            }
        )
        .where_records_ordered_by("global_cited_by_highest")
        .run()
    )


    print(subject_area, frame_by_area.shape[0])

In [12]:
# grey colors: https://www.w3schools.com/colors/colors_shades.asp
from techminer2.packages.correlation.cross import NetworkMapPlot  # type: ignore

plot = (
    NetworkMapPlot()
    #
    # FIELD:
    .with_field("subject_areas")
    .having_terms_in_top(20)
    .having_terms_ordered_by("OCC")
    .having_term_occurrences_between(None, None)
    .having_term_citations_between(None, None)
    .having_terms_in(None)
    #
    # CROSS WITH:
    .with_other_field("raw_source_title")
    #
    .with_correlation_method("pearson")
    #
    # NETWORK:
    .using_spring_layout_k(0.7)
    .using_spring_layout_iterations(60)
    .using_spring_layout_seed(2)
    #
    .using_edge_colors(["#7793a5", "#7793a5", "#7793a5", "#7793a5"])
    .using_edge_similarity_threshold(0)
    .using_edge_top_n(None)
    .using_edge_widths([1, 1, 2, 3])
    #
    .using_node_colors(["#7793a5"])
    .using_node_size_range(10, 50)
    #
    .using_textfont_opacity_range(0.5, 1.0)
    .using_textfont_size_range(8, 13)
    #
    .using_xaxes_range(None, None)
    .using_yaxes_range(None, None)
    .using_axes_visible(False)
    #
    # DATABASE:
    .where_root_directory_is("../")
    .where_database_is("main")
    .where_record_years_range_is(None, None)
    .where_record_citations_range_is(None, None)
    .where_records_match(None)
    #
    .run()
).update_layout(width=1000, height=900)
plot.write_html("../reports/sec.4.2.corr_map.subject_areas.html")
plot
